In [1]:
# 按照逻辑回归的假设产生数据
import numpy as np


# 类别1的个数
n = 1000
# 类别0的倍数（相比于类别1）
zero_times = 20
np.random.seed(4060)
cov = [[1, 0], [0, 1]]
X = np.random.multivariate_normal([0, 0], cov, 2 * n * zero_times)
beta = np.array([1, -1]).reshape(2, 1)
## 产生随机扰动项
error = np.random.logistic(size=2 * n * zero_times).reshape(-1, 1)
Y = (np.dot(X, beta) + error > 0) + 0

In [2]:
# 产生非均衡数据集
X1 = X[np.where(Y == 1)[0]][:n]
Y1 = Y[np.where(Y == 1)[0]][:n]
X0 = X[np.where(Y == 0)[0]]
Y0 = Y[np.where(Y == 0)[0]]
X = np.append(X0, X1, axis=0)
Y = np.append(Y0, Y1, axis=0)
print("Y = 1的个数：%s" % Y[np.where(Y == 1)].shape)
print("Y = 0的个数：%s" % Y[np.where(Y == 0)].shape)

Y = 1的个数：1000
Y = 0的个数：20000


In [3]:
# 定义模型的评估方法
from sklearn import metrics


def evaluate_model(Y, pred):
    """
    评估模型效果，其中包括ACC，AUC以及预测结果中类别1的个数
    """
    print("数据中Y = 1的个数：%s" % Y[np.where(Y == 1)].shape)
    print("预测结果中Y = 1的个数：%s" % pred[np.where(pred == 1)].shape)
    print("模型准确度：%s" % metrics.accuracy_score(Y, pred))
    fpr, tpr, _ = metrics.roc_curve(Y, pred)
    print("模型AUC：%s" % metrics.auc(fpr, tpr))

In [4]:
# 直接训练模型得到结果
from sklearn.linear_model import LogisticRegression


## 为了消除惩罚项的干扰，将惩罚系数设为很大
model = LogisticRegression(C=1e4)
model.fit(X, Y.ravel())
pred = model.predict(X)
evaluate_model(Y, pred)

数据中Y = 1的个数：1000
预测结果中Y = 1的个数：42
模型准确度：0.9525714285714286
模型AUC：0.511025


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [5]:
# 修正类别权重训练模型得到结果
positive_weight = len(Y[Y > 0]) / float(len(Y))
## 用类别占比的倒数作为类别权重，使数据集“回归均衡”
class_weight = {1: 1. / positive_weight, 0: 1. / (1 - positive_weight)}
## 为了消除惩罚项的干扰，将惩罚系数设为很大
model = LogisticRegression(class_weight=class_weight, C=1e4)
model.fit(X, Y.ravel())
pred = model.predict(X)
evaluate_model(Y, pred)

数据中Y = 1的个数：1000
预测结果中Y = 1的个数：6347
模型准确度：0.7197619047619047
模型AUC：0.7251


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
